# **Programmering och Integraler**







# **1. Rotationsvolym**
Vi kommer undersöka rotationsvolymer och hur dessa kan visualisas och beräknas med hjälp av programmering.

> **Uppgift 1.0:** vad är en rotationsvolym för någonting? Diskutera med din programmeringskompis.



### **1.1. Kod till problemet**


Koden nedan har skapats för att i slutändan rita ut rotationsvolymer av funktioner som du själv bestämmer. Merparten av den kod som krävs för att rita upp rotationsvolymerna har dolts i kommandon som ni kan använda för att kunna fokusera på *matematiken* istället för koden. 

Vi kommer att gå igenom de kommandon som ni ska använda steg för steg och det finns utrymme för egen experimentation.

**Kör koden i rutan nedan för att resten av programmet ska fungera.**

In [ ]:
#@title Kod för att senare kunna rita rotationsvolymer [Kör]
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from IPython.display import clear_output
import scipy.integrate as sc
pio.renderers.default = "colab"

lower_lim, upper_lim = 0, 0
default_lim = 0


# Kan användas för att ändra på integrationsgränserna i x-led. Default är a=0 och b=10
def set_x_lims(lower_limit=0, upper_limit=10):
    global lower_lim, upper_lim
    lower_lim = lower_limit
    upper_lim = upper_limit


def least_of_two(a, b):
    if a < b:
        return a
    else:
        return b


def largest_of_two(a, b):
    if a < b:
        return b
    else:
        return a


fig = make_subplots(rows=1, cols=1,
                    specs=[[{'is_3d': True}]],
                    # subplot_titles=['Rotationsvolym kring x-axeln']
                    )


def draw_radial_contour(function, num_of_traces=8, color="#e89938"):
    x = np.linspace(lower_lim, upper_lim, 100)
    v_step = (2 * np.pi) / num_of_traces    # andel av varv mellan varje konturlinje
    v = v_step/2
    while v <= 2 * np.pi:
        fig.add_trace(go.Scatter3d(
            x=x, y=function(x) * np.cos(v), z=function(x) * np.sin(v),
            marker=dict(size=0.1, color=color),
            line=dict(color=color, width=2), showlegend=False))
        v += v_step


def draw_radial_contours(function_a, function_b, num_of_traces=8):
    draw_radial_contour(function_a, num_of_traces)
    draw_radial_contour(function_b, num_of_traces)


def draw_x_rotation(function, x_start=lower_lim, x_stop=upper_lim, show_contours=True, num_contour=8, contour_color="#3b10a1"):
    if x_start is default_lim and x_stop is default_lim:
        x_start, x_stop = lower_lim, upper_lim
    u, v = np.mgrid[x_start:x_stop:100j, 0:4 * np.pi:200j]      # Problem konturlinjer löses med rotation två varv
    x0 = u
    y0 = (function(u)) * np.cos(v)
    z0 = (function(u)) * np.sin(v)

    fig.add_trace(
        go.Surface(
            x=x0, y=y0, z=z0,  opacity=0.1, surfacecolor=x0, showlegend=False,   # Färg varierar längs x-axeln
            contours={
                "x": {"show": show_contours, "start": x_start, "end": x_stop + (x_stop-x_start)/(8*num_contour), 
                      "size": 2*(x_stop-x_start)/num_contour, "color": contour_color},
            }
        ))
    if show_contours:
        draw_radial_contour(function, num_of_traces=num_contour, color=contour_color)


def draw_function(function, show_area=True, x_start=lower_lim, x_stop=upper_lim, resolution=100):
    if x_start is default_lim and x_stop is default_lim:
        x_start, x_stop = lower_lim, upper_lim
    x = np.linspace(x_start, x_stop, resolution)
    fig.add_trace(go.Scatter3d(
        x=x, y=abs(function(x)), z=np.linspace(0, 0, resolution),
        marker=dict(size=0.1, color='magenta'),
        line=dict(color='red', width=2), showlegend=False
    ))
    if show_area:
        u, v = np.mgrid[x_start:x_stop:100j, 0:np.pi/2:100j]
        fig.add_trace(go.Surface(
            x=u, y=abs(function(u))*np.cos(v), z=0*u, surfacecolor=0*u, opacity=0.5, showlegend=False, showscale=False
        ))


def draw_functions(function_a, function_b, show_area=True, x_start=lower_lim, x_stop=upper_lim, resolution=100):
    if x_start is default_lim and x_stop is default_lim:
        x_start, x_stop = lower_lim, upper_lim
    x = np.linspace(x_start, x_stop, resolution)
    fig.add_traces([
        go.Scatter3d(
            x=x, y=abs(function_a(x)), z=np.linspace(0, 0, resolution),
            marker=dict(size=0.1, color='magenta' ),
            line=dict(color='red', width=2), showlegend=False),
        go.Scatter3d(
            x=x, y=abs(function_b(x)), z=np.linspace(0, 0, resolution),
            marker=dict(size=0.1, color='magenta' ),
            line=dict(color='red', width=2), showlegend=False
        )],
        rows=[1, 1], cols=[1, 1]
    )
    if show_area:      
        u, v = np.mgrid[x_start:x_stop:100j, 0:np.pi/2:100j]
        fig.add_trace(go.Surface(
            x=u, y=(abs(function_a(u))-abs(function_b(u)))*np.cos(v)+abs(function_b(u)),
            z=0*u, opacity=0.5, showlegend=False, showscale=False
        ))


def draw_cross_section(function, x_val=upper_lim):
    if x_val is default_lim:
        x_val = int(upper_lim/4)
    radius = function(x_val)
    u, v = np.mgrid[0:radius:100j, 0:2*np.pi:100j]
    angle = np.linspace(0, 2*np.pi, 100)
    x = np.linspace(x_val, x_val, 100)
    fig.add_traces([
        go.Surface(
            x=0*u+x_val, y=u*np.cos(v), z=u*np.sin(v), opacity=0.5, showlegend=False, surfacecolor=0*u,  showscale=False),
        go.Scatter3d(
            x=x, y=function(x)*np.cos(angle), z=function(x)*np.sin(angle),
            marker=dict(size=0.1, color='magenta'),
            line=dict(color='red', width=2), showlegend=False)
    ],
        rows=[1, 1], cols=[1, 1]
    )


def draw_cross_section_between(function_a, function_b, x_val=upper_lim, opacity=0.2):
    if x_val is default_lim:
        x_val = int(upper_lim/2)
    radius_upper = largest_of_two(abs(function_a(x_val)), abs(function_b(x_val)))
    radius_lower = least_of_two(abs(function_a(x_val)), abs(function_b(x_val)))
    u, v = np.mgrid[radius_lower:radius_upper:100j, 0:2*np.pi:100j]
    angle = np.linspace(0, 2*np.pi, 100)
    x = np.linspace(x_val, x_val, 100)
    fig.add_traces([
        go.Surface(
            x=0*u+x_val, y=u*np.cos(angle), z=u*np.sin(v),
            surfacecolor=0*u, opacity=opacity, showlegend=False, showscale=False),
        go.Scatter3d(
            x=x, y=function_a(x)*np.cos(angle), z=function_a(x)*np.sin(angle),
            marker=dict(size=0.1, color='magenta'),
            line=dict(color='red', width=2), showlegend=False),
        go.Scatter3d(
            x=x, y=function_b(x) * np.cos(angle), z=function_b(x) * np.sin(angle),
            marker=dict(size=0.1, color='magenta'),
            line=dict(color='red', width=2), showlegend=False
        )
    ],
        rows=[1, 1, 1], cols=[1, 1, 1]
    )


def draw_multiple_cs_between(function_a, function_b, num_of_sections=4):
    step = (upper_lim - lower_lim) / num_of_sections
    curr_section = lower_lim + step/2
    while curr_section <= upper_lim:
        draw_cross_section_between(function_a, function_b, curr_section, opacity=0.4)
        curr_section += step


def draw_axis(max_value=5):  
    x0 = np.linspace(0, 0, 100)
    y0 = np.linspace(0, 0, 100)
    z0 = np.linspace(0, 0, 100)
    x1 = np.linspace(least_of_two(lower_lim - 1, -1), upper_lim + 1, 100)  # x-värden för x-axel
    y1 = np.linspace(-abs(max_value), abs(max_value), 100)  # y-värden för y-axel
    z1 = np.linspace(-abs(max_value), abs(max_value), 100)  # för z-axel

    fig.add_traces([go.Scatter3d(
        x=x1, y=y0, z=z0,
        marker=dict(size=0.1, color='magenta'),
        line=dict(color='black', width=4), showlegend=False),
        go.Scatter3d(
        x=x0, y=y1, z=z0,
        marker=dict(size=0.1, color='magenta'),
        line=dict(color='black', width=4), showlegend=False),
        go.Scatter3d(
            x=x0, y=y0, z=z1,
            marker=dict(size=0.1, color='magenta' ),
            line=dict(color='black', width=4), showlegend=False
        )],
        rows=[1, 1, 1],
        cols=[1, 1, 1]
    )


def clear_figure():
    fig.data = []


fig.update_layout(width=1200, height=700)  # Uppdatera storlek för figuren
# Ändra utgångsläge för kamera 
fig.layout.scene = dict(camera=dict(eye=dict(x=0, y=0, z=2), up=dict(x=1, y=0, z=0)), dragmode='turntable')


Nedan följer en kodruta som ni kommer att arbeta med. Ni kommer stegvis  implementera flera kommandon som har skapats i den dolda koden.

---

>**Uppgift 1.1.1:** Kör koden i rutan nedan för att rita ut ett koordinatsystem i 3 dimensioner.

---

>**Uppgift 1.1.2:** Vad gör funktionen ***set_x_lims()*** respektive ***draw_axis()***? Diskutera med din programmeringskompis, testa olika värden samt att kommentera bort funktionerna.

---

In [ ]:
clear_figure()    # Rensa utdata från tidigare körningar

a = 0
b = 1
set_x_lims(a, b)  
draw_axis(max_value=2) 

fig.show()    # Visa figuren

### **1.2 Integraler i 2D**
I nedanstående kod har vi definierat en funktion $f(x)=0,5x^2$ samt de två gränsvärdena $a$ och $b$. Vi ritar därefter ut funktionen med ***draw_function(f)***. 

**OBS:** som koden ser ut nu så ritar vi alltid grafen till $|f(x)|$.

---

> **Uppgift 1.2.1:** Testa att ändra funktionen $f(x)$ till några av följande:
* $f(x)= 2\cdot x^0, a=0, b=2$
* $f(x)=x/2, a=0, b=4$
* $f(x)=2\cdot \cos(x)$, $a=0$, $b=\pi/2$. OBS: $cos(x)$ skrivs i koden ***np.cos(x)*** och $\pi$ skrivs ***np.pi***. 

---


In [ ]:
clear_figure()    # Rensa utdata från tidigare körningar

# --------------------------------------------------------------- #
# Ändra här
def f(x):
    return 0.5*x**2  # f(x)=0.5x^2. Exponenter skrivs som a**b i Python. Multiplikationer måste skrivas ut.
# --------------------------------------------------------------- #

a = 0
b = 2
set_x_lims(a, b)
draw_axis(max_value=2)

draw_function(f)  # Ritar funktion samt arean mellan dess graf och x-axeln

fig.show()

---

>**Uppgift 1.2.2:** Definiera en ny funktion, $g(x)$, på samma sätt som ovan. Använd kommandot ***draw_functions(func_a, func_b)*** för att rita ut arean mellan dina två funktioner.

**OBS:** kodcellen kommer ej kunna exekveras om $g(x)$ inte är definierad.

---

In [ ]:
clear_figure()    # Rensa utdata från tidigare körningar


def f(x):
    return x ** 2

# --------------------------- #
# Definiera g(x) här

# --------------------------- #

a = 0
b = 1
set_x_lims(a, b)
draw_axis(max_value=1.5)

draw_functions(f, g)

fig.show()

---

>**Uppgift 1.2.3:** Hur bestämmer vi integralen mellan två funktioner? Diskutera med din programmeringskompis. Om tid finns, beräkna algebraiskt en integral mellan två valfria funktioner och gränser.

---
(Kodcellen nedan skapar läraren) Följa-John kodning som gör så att vi kan beräkna integralen mellan två funktioner.


In [ ]:
# Implementation av calculate_integral().
def calculate_integral(function, a, b):
    integral, err = sc.quad(function, a, b)
    return integral

#-------------------------------------------
# Definera en funktion som ska integreras
def diff_func(x):
    return abs(f(x)-g(x))

#-------------------------------------------

calculate_integral(diff_func, a, b)

## **1.3 Rotationsvolymer**
Vi kommer nu att gå över från 2D till 3D och rita rotationsvolymer.
### **1.3.1 Visualisera rotationsvolymer**
Innan vi ritar rotationsvolymer med hjälp av datorn kommer vi att använda kraften av hjärnan, papper och penna.

---

>**Uppgift 1.3.1**: Hur ser rotationsvolymen för följande funktioner ut? Försök att skissa och diskutera med din programmeringskompis.
* $f(x)=2\cdot x^0, a=0, b=2$

>* $g(x)=x, a=0, b=2$

>* $h(x)=f(x)-g(x), a=0, b=2$, med $f(x), g(x)$ enligt ovan.

---

### **1.3.2 Rita rotationsvolymer**
Vi kommer nu att använda datorn och lite programmering för att hjälpa oss visualisera rotationsvolymer. Inledningsvis kommer vi kolla på rotationsvolymen till en funktion.

---

>**Uppgift 1.3.2** 
Ändra funktionerna $f(x)$ för att rita rotationen av denna kring $x$-axeln. Rita rotationerna för funktionerna som introducerades i *uppgift 1.3.1* eller för egna funktioner som ni väljer.

---

In [ ]:
clear_figure()    # Rensa utdata från tidigare körningar

# ---------------------------------- #
# Ändra funktionerna nedan
def f(x):
    return 2*x**0

# Ändra gränserna a, b nedan
a = 0
b = 4
# ----------------------------------- #

set_x_lims(a, b)
draw_axis(max_value=3)

draw_function(f)
draw_x_rotation(f, num_contour=8)  # Ritar en surface-plot på när en funktion roteras kring x-axeln

fig.show()

---

> **Uppgift 1.3.3** 

* Ändra funktionerna $f(x)$ och $g(x)$ för att rita rotationen av dessa runt $x$-axeln. Prova exempelvis följande par av funktioner:

>>* $f(x)=x^2, g(x)=x, a=0, b=1$
>>* $f(x)=2\cos(x), g(x)=(e^x-1)/(2\pi), a=0, b=1.345 $
>> OBS: $e^x$ skrivs ***np.exp(x)*** i Python.
>* Använd kommandot **draw_cross_section_between(f, g, c)** för att se tvärsnittet mellan rotationerna av $f(x)$ och $g(x)$ vid punkten $x=c$. $c$ ersätts av numret för det x-värde som du vill rita tvärsnittet för. 

---


In [ ]:
clear_figure()    # Rensa utdata från tidigare körningar


def f(x):
    return np.sin(x)

def g(x):
    return np.sin(x)/3

a = 0
b = np.pi
set_x_lims(a, b)
draw_axis(max_value=1)

# draw_functions(f, g)
draw_x_rotation(f, show_contours=False)
draw_x_rotation(g, show_contours=False)
## ------------------------------------- #
# Använd kommandot draw_cross_section_between(f, g, c) för att se tvärsnittet
c = 1

## ------------------------------------- #

fig.show()

---

**Uppgift 1.3.4:** 

>* Ändra funktionerna $f(x)$ och $g(x)$ för att rita rotationerna av dessa kring $x$-axeln. Variera antalet tvärsnitt genom att ändra på parametern ***num_of_sections*** i kommandot ***draw_multiple_cs_between(f, g, num_of_sections)***.

>* Bestäm en formel för arean för varje tvärsnitt. Går det att generalisera till godtyckliga funktioner $f(x)$ och $g(x)$?

---



In [ ]:
clear_figure()    # Rensa utdata från tidigare körningar


def f(x):
    return x**0

def g(x):
    return x

a = 0
b = 1
set_x_lims(a, b)
draw_axis(max_value=1)

draw_functions(f, g)
draw_x_rotation(f, show_contours=False)
draw_x_rotation(g, show_contours=False)

# ----------------------------------------------------------- #
# Använd kommandot draw_multiple_cs_between(f, g, num) här
draw_multiple_cs_between(f, g, num_of_sections=2)
# ----------------------------------------------------------- #

fig.show()

Sammanfattning av **uppgift 1.3.4** i helklass, genomgång av hur arean kan bestämmas för varje tvärsnitt.

---

>**Uppgift 1.3.5:** Diskutera med din programmeringskompis: hur kan värdet på rotationsvolymen bestämmas genom arean för tvärsnittet?

---

Sammanfattning i helklass.

---

>**Uppgift 1.3.6:** 
* Skriv klart funktionen ***area_func(x)*** som representerar arean av ett godtyckligt tvärsnitt som funktion av $x$. 

>* Räkna ut skillnaden i volym mellan rotationskropparna som uppstår när vi roterar funktionerna $f(x)$ och $g(x)$ runt $x$-axeln. Använd resultatet från föregående uppgift. Du kan kan exempelvis testa några av följande funktioner:

> > * $f(x)=2\cdot x^0, g(x)=x, a=0, b=2$

> > * $f(x)=x^2, g(x)=x, a=1, b=3$

> > * $f(x)=2\cdot\cos(x), g(x)=\sin(x), a=0, b=\arctan(2)$. OBS: kom ihåg att $\cos(x)$ skrivs ***np.cos(x)*** och så vidare.

---

>**Uppgift 1.3.7:** Hur kan vi göra för att räkna ut rotationsvolymen för en funktion $f(x)$ vid rotation kring $x$-axeln? Diskutera med din programmeringskompis.

---

In [ ]:
clear_figure()    # Rensa utdata från tidigare körningar

def f(x):
    return x**0

def g(x):
    return x*0

# ---------------------------------------------------------------- #
# Arbeta med area_func(x)
def area_func(x):
    area_cross_section = 0   # Ändra 0 till uttryck för area tvärsnitt
    return area_cross_section
# ---------------------------------------------------------------- #

a = 0
b = 1
set_x_lims(a, b)
draw_axis(max_value=2)

draw_x_rotation(f, show_contours=False)
draw_x_rotation(g, show_contours=False)
draw_multiple_cs_between(f, g, 8)

# ------------------------------------------------------------ #
# Ändra koden här - beräkna volymen med hjälp av calculate_integral(h, a, b).
# Ersätt h med areafunktionen som du precis definierade.
volume = 0
# ------------------------------------------------------------ #
print("Volymen blir:", volume)

fig.show()

# **2. Sammanfattning**

---

>**Uppgift 2.1:** Diskutera följande frågor med din programmeringskompis:
>* Vad har vi gjort idag?
>* Vilken matematik har vi fokuserat på under dagens lektion?
>* På vilket/vilka sätt har vi haft användning av programmeringen?

---

Sammanfattning i helklass. Kort info om enkät och intervju.



---

>**Uppgift 2.2:** Fyll i [**följande enkät**](https://forms.office.com/Pages/ResponsePage.aspx?id=Vh4-oU-JMEaOj-i0ZhfqZ9QPEYEQpElJli24qTWd9gpUNENCRjBDWFk5WjkxVk9YVjFWTkxaV0lWMC4u) som handlar om vad du tycker om dagens lektion.

---

Tack för att ni deltog på denna lektion!

# **3. Extra (i mån av tid): Acceleration och databehandling**
I detta kapitel kommer vi istället använda programmering för att hantera stora mängder data som har mätts med hjälp av accelerometern i en smartphone. Vanligtvis använder din mobil detta för att autorotera skärmen. 

## **3.1 Koppling till fysiken**
Vi kommer som sagt att arbeta med acceleration, vilket är något som ni antagligen känner igen från fysiken.

---

>**Uppgift 3.1.1:** till vad kan vi använda data för acceleration och tid som accelerometern i din mobil mäter? Fundera själv och diskutera sedan med din programmeringskompis.

---

## **3.2 Diskreta funktioner**
Alla **digitala** sensorer arbetar diskret och mäter data för (i vårt fall) specifika tidpunkter. I den kommande uppgiften är frekvensen för denna datamätning i snitt $500$ Hz vilket innebär en medeltid mellan datapunkter på $2$ millisekunder. Mellan dessa tidpunkter vet vi inte vilka värden det som vi mäter egentligen har, men rimligtvis så är det antingen ett värde mellan två på varandra följande datapunkter eller relativt nära dessa. 

En diskret funktion är en funktion vars definitionsmängd är diskret. Ett klassiskt exempel på en diskret funktion är priset på äpplen som funktion av ***antalet*** medan en kontinuerlig motsvarighet är priset på äpplen som funktion av ***vikten***. Den viktiga skillnaden är att definitionsmängden ("tillåtna" x-värden) för en diskret funktion är skilda punkter istället för "alla" x-värden.

---

>**Uppgift 3.2.1:** Hur beräknar man integralen för en diskret funktion? Diskutera med din programmeringskompis.

---

Sensorer är aldrig helt exakta, även om de kan ha mycket hög precision. Accelerometern i din smartphone mäter acceleration i tre dimensioner och känner hela tiden av gravitionskraften vilket gör den mycket känslig för hur mobilen är vinklad. 

Resultat från datainsamlingen kommer därför inte att vara helt exakta på grund av följande anledningar:

* Vinkeln på mobilen
* Mätfel hos sensorn
* Vibrationer och högfrekvent brus




## **3.3 Data och beräkning för hand**##

Vi kommer nu att titta på data som har mätts upp med en accelerometer i en smartphone.

---

>**Uppgift 3.3.1:** Ladda upp filen i Google colaboratory så som visats av läraren. Hur många mätningar innehåller filen?

---

>**Uppgift 3.3.2:** Uppskatta hur lång tid det hade tagit att beräkna integralen för hand med paralleltrapets-metoden. Vilka antaganden gör du?

---

Upprepade beräkningar är något som datorer är mycket bättre på än människor. 

## **3.4 Behandla data med programmering**
För att senare kod skall fungera kommer ni att behöva köra koden i cellen nedan. Likt avsnitt **2. Rotationsvolym** så behöver ni inte förstå någon del av denna dolda kod men vi kommer att använda metoder i Python som definieras i den för att läsa in och rita upp data.

In [ ]:
# @title Kod med kommandon som behövs senare [Kör cellen]
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import math as mth
import scipy.integrate as sc
from google.colab import files

latest_y_vals = np.array([])
times_called = 0

# Funktion som läser filen och returnerar en tuple (tänk lista på värden) med datavärden
# param: filnamn (format viktigt); output = fem listor med värden


def load_data(filename):
    f = open(filename, "r", encoding="utf8")

    on_first_line = True

    t = np.array([])
    a_x = np.array([])
    a_y = np.array([])
    a_z = np.array([])
    a_tot = np.array([])

    for x in f:
        num_words = 0
        if on_first_line:
            on_first_line = False
            continue
        curr_line = x
        line_sub = ""
        for letter in curr_line:
            if letter == " " or letter == "\t" or letter == "\n":  # filen verkar använda \t mellan talen och \n för ny rad
                num_words += 1
                if num_words == 1:
                    t = np.append(t, float(line_sub))
                elif num_words == 2:
                    a_x = np.append(a_x, float(line_sub))
                elif num_words == 3:
                    a_y = np.append(a_y, float(line_sub))
                elif num_words == 4:
                    a_z = np.append(a_z, float(line_sub))
                elif num_words == 5:
                    a_tot = np.append(a_tot, float(line_sub))
                else:
                    continue
                line_sub = ""  # Nollställ efter sparad i array

            elif letter == ",":
                line_sub += "."
            else:
                if line_sub == "−":  # samma som minustecken i filen i fråga
                    line_sub = "-"  # annars blir det fel minustecken och float() funkar ej
                line_sub += letter

    return t, a_x, a_y, a_z, a_tot


# Tar ett "moving average", dvs medel av num antal datapunkter. Minskar ej upplösning av data
def moving_avg_filter(arr, num=125):
    temp_arr = np.array([])
    avgd_arr = np.array([])

    for elem in arr:
        temp_arr = np.append(temp_arr, elem)
        if len(temp_arr) > num:  # Fall: vi har kommit upp i num antal
            temp_arr = temp_arr[1:]
        avgd_arr = np.append(avgd_arr, np.mean(temp_arr))

    return avgd_arr


# Långsammare variant av integral()-se nedan-, innan optimering.
def slow_integral(array_to_integrate, array_integrand):
    integ = np.array([])
    for i in range(1, len(array_to_integrate) + 1):
        integ = np.append(integ, np.trapz(array_to_integrate[0:i], array_integrand[0:i]))
    # print("Size input:", len(array_to_integrate), "Size output:", len(integ))
    return integ


# Approximerar integralen utifrån datapunkter i array. Summerar approximationer mellan två punkter
# Param: två arrayer/listor - den första med funktionens värden, den andra med integrandens värden
# Output: en array av samma storlek som approximativt motsvarar primitiva funktionen
def integral(array_to_integrate, array_integrand):
    integ = np.array([])
    acc_sum = 0
    integ = np.append(integ, acc_sum)
    for i in range(1, len(array_to_integrate)):
        area_dt = np.trapz(array_to_integrate[i - 1:i + 1], array_integrand[i - 1:i + 1])
        acc_sum += area_dt
        integ = np.append(integ, acc_sum)
    return integ


def gforce_to_ms2(arr):
    return arr * 9.82


# Hjälpfunktion till draw_subplot(). Returnerar aktuell färg.
def get_color(num):
    colors = ['blue', 'magenta', 'orange']
    try:
        return colors[num]
    except IndexError:
        return "blue"


def draw_current_subplot(num, x_values, y_values, xlabel="Tid [$s$]", ylabel="Acceleration/Hastighet/Sträcka"):
    if total_plots == 1:
        axis.plot(x_values, y_values, color=get_color(num))
        axis.grid()
        axis.set(ylabel=ylabel, xlabel=xlabel)
    else:
        axis[num].plot(x_values, y_values, color=get_color(num))
        axis[num].grid()
        axis[num].set(ylabel=ylabel, xlabel=xlabel)


# Förenklar antagligen användandet av matplotlibs subplots. Ritar ut en subplot förutsatt att figuren har skapats med "fig, axis = plt.subplots(x, 1)" först har skapats.
# Param: x-värden i lista/array, y-värden i lista/array. Valfri param: ylabel, titeln på y-axeln
# Output: ingen. Ritar funktionen i aktuell subplot
def draw_subplot(x_values, y_values, xlabel="Tid [$s$]", ylabel="Acceleration/Hastighet/Sträcka"):
    global times_called, latest_y_vals
    try:
        draw_current_subplot(times_called, x_values, y_values, xlabel=xlabel, ylabel=ylabel)
        if not (y_values is latest_y_vals):
            times_called += 1
        latest_y_vals = y_values
    except NameError:
        times_called = 0
        draw_current_subplot(times_called, x_values, y_values, xlabel=xlabel, ylabel=ylabel)
        latest_y_vals = y_values
        times_called += 1
    except IndexError:
        times_called = 0
        draw_current_subplot(times_called, x_values, y_values, xlabel=xlabel, ylabel=ylabel)
        latest_y_vals = y_values
        times_called += 1


def get_error(arr, time_arr, time, resolution=500):
    step = int(resolution/10)
    index = 0
    for i in range(0, len(arr), step):
        if time_arr[i] >= time:
            index = i
            break
    err = np.mean(arr[:index])
    return err


def clear_graph():
    global times_called, latest_y_vals
    times_called = 0
    latest_y_vals = np.array([])

--- 

> **Uppgift 3.4.1:** Kopiera filsökvägen till textfilen som ni tidigare laddade upp (högerklicka på filen -> "kopiera sökväg"). Läs in accelerationsdata genom att ändra ***file_path*** i funktionen ***load_data("file_path")*** till sökvägen.

Koden kommer även att rita ut den uppmätta accelerationen i y-led med kommandot ***draw_subplot(t, acceleration)***.

---

In [ ]:
# ----------------------------------------------------------------------------------------------------- #
# Ändra filnamn i raden under
t, a_x, a_y, a_z, a_tot = load_data("file_path") #läs in data från filen, lagra i variabler
# ----------------------------------------------------------------------------------------------------- #

# Rita subplots för data
total_plots = 1   
fig, axis = plt.subplots(total_plots, figsize=(16,8))

acceleration = a_y * 9.82  # Acceleration i y_led är den som vi är ute efter
draw_subplot(t, acceleration, ylabel="Acceleration [$m/s^2$]")  # Ritar ut accelerationen som funktion av tiden

plt.show()


---

> **Uppgift 3.4.2:** Integrera hastigheten (***velocity***) för att få fram sträckan med hjälp av ***integral(func, t)***. Rita grafen för alla tre storheter genom att använda kommandot ***draw_subplot()*** tre gånger. 

> Med tanke på graferna för acceleration, hastighet och sträcka: vad tror du händer i verkligheten? Diskutera med din programmeringskompis och motivera ditt svar.

---

In [ ]:
total_plots = 3   # Anger antal subplots
fig, axis = plt.subplots(total_plots, figsize=(20,10))
clear_graph()

acceleration = a_y * 9.82
velocity = integral(acceleration, t)  # Integralen av accelerationen = hastigheten
# ------------------------------------------------------------- #
# Beräkna sträckan med hjälp av integral()-kommandot
distance = 0
# ------------------------------------------------------------- #

draw_subplot(t, acceleration, ylabel="Acceleration [$m/s^2$]")

# ------------------------------------------------------------- #
# Rita ut hastigheten genom att använda kommandot draw_subplot

# Rita ut sträckan genom att använda kommandot draw_subplot

# ------------------------------------------------------------- #

plt.show()

**Läraren förklarar hur data samlades in**